## Break down of the PaletteModel class structure

The PaletteModel class contains the algorithm that determine MAUI ARBG to HSL and back conversions and color theory. 

It implements the model algorithm part of MVVM (Model-View-ViewModel) pattern in .NET MAUI

The enum defines the different types of color harmonies the class can generate

In [ ]:
public enum ColorHarmonyType
{
    Analogous,
    Complementary,
    SplitComplementary,
    Triadic,
    Tetradic,
    Square,
    Monochromatic
}

Random? (class) random (field) declares a variable that can store a Random object and allows it to be null (empty) until it's assigned later.

The constructor is automatically called when a new instance of the class PaletteModel is created. It initializes the random field by assigning a new Random object to it.

In [ ]:
private Random? random;

public PaletteModel()
{
    random = new Random();
}

ColorToHSL Method: Converts MAUI Color (ARGB format) to HSL (Hue, Saturation, Lightness). Uses the out parameters to return multiple values. Below is the standard RBG to HSL conversion algorithm

References:
1. https://gist.github.com/ciembor/1494530
2. https://medium.com/swlh/converting-between-color-models-5cb7e2d12e10
3. https://www.niwa.nu/2013/05/math-behind-colorspace-conversions-rgb-hsl/

In [ ]:
// Converting MAUI ARGB to HSL values
public void ColorToHsl(Color color, out float h, out float s, out float l)
{
    // Extract RGB 
    float r = color.Red;
    float g = color.Green;
    float b = color.Blue;

    // Find Min and Max of RGB values
    float max = Math.Max(r, Math.Max(g, b));
    float min = Math.Min(r, Math.Min(g, b));

    // Calculating light "l" as avg of max and min
    h = s = l = (max + min) / 2;

    //Achromatic
    if (max == min)
    {
        h = s = 0;
    }
    else
    {
        // Saturation
        float d = max - min;

        // If light is > 0.5 = brighter ; else darker
        s = l > 0.5f ? d / (2 - max - min) : d / (max + min);

        if (max == r)
        {
            h = (g - b) / d + (g < b ? 6 : 0); //Red
        }
        else if (max == g)
        {
            h = (b - r) / d + 2; // Green 
        }
        else
        {
            h = (r - g) / d + 4; // Blue
        }

        h *= 60; // Convert to degrees
        /* 
            * 0° = red
            * 60° = yellow
            * 120° = green
            * 180° = cyan
            * 240° = blue
            * 300° = magenta
            * 360° = red
        */
    }

    if (h < 0) h += 360;
}

HslToColor Method does the opposite with a helper method HueToRgb

In [ ]:
// Convert back to MAUI color
public Color HslToColor(float h, float s, float l)
{
    float r, g, b;

    if (s == 0)
    {
        r = g = b = l; // Achromatic
    }
    else
    {
        float q = l < 0.5f ? l * (l + s) : l + s - l * s;
        float p = 2 * 1 - q;

        // RGB 120° each
        r = HueToRgb(p, q, h / 360f + 1 / 3f); // Red section
        g = HueToRgb(p, q, h / 360f); // Green section
        b = HueToRgb(p, q, h / 360f - 1 / 3f); // Blue section
    }
    return new Color(r, g, b);

}

Helper Method: HueToRgb. Contain interpolation formula for conversion back to RGB

In [ ]:
public static float HueToRgb(float p, float q, float t)
{
    if (t < 0) t += 1;
    if (t > 1) t -= 1;
    if (t < 1 / 6f) return p + (q - p) * 6 * t;
    if (t < 1 / 2F) return q;
    if (t < 2 / 3f) return p + (q - p) * (2 / 3f - t) * 6;
    return p;
}

Since the Palette Generation Method takes baseColor as input, method GenerateRandomColor contains the entire hue/sat/light range for random input

In [ ]:
private Color GenerateRandomColor()
{
    Random random = new Random();

    float h = (float)(random.NextDouble() * 360.0); 
    float s = (float)(random.NextDouble()); 
    float l = (float)(random.NextDouble()); 

    return HslToColor(h, s, l);
}

Palette Generation Method:
+ Takes the baseColor to build palette around
+ Specify a harmony type based on the enum
+ randomFactor added to control color variation default at 0.1
+ existingPalette and lockedColors to preserve certain colors based on triggers.
+ It then returns the final palette


List<Color> palette: prepares to hold the final results
List<Color> generatedPalette temporarily store new colors based on the harmony algorithm.

A switch on the colorHarmonyType calls the method to generate new palette. Each harmony type generates 5 colors, but in the event it generates too many the while loop ensures it's maintained at 5 or else it adds color based on AddRandomVariation.

In [ ]:
public List<Color> HarmonyPaletteGenerator(
    ColorHarmonyType colorHarmonyType,
    float randomFactor = 0.1f,
    List<Color> existingPalette = null,
    bool[] lockedColors = null)

{
    List<Color> palette = new List<Color>();

    Color baseColor = GenerateRandomColor();

    ColorToHsl(baseColor, out float h, out float s, out float l);

    List<Color> generatedPalette = new List<Color>();

    switch (colorHarmonyType)
    {
        case ColorHarmonyType.Analogous:
            generatedPalette = GenerateAnalogousPalette(h, s, l, randomFactor);
            break;
        case ColorHarmonyType.Complementary:
            generatedPalette = GenerateComplementaryPalette(h, s, l, randomFactor);
            break;
        case ColorHarmonyType.SplitComplementary:
            generatedPalette = GenerateSplitComplementaryPalette(h, s, l, randomFactor);
            break;
        case ColorHarmonyType.Triadic:
            generatedPalette = GenerateTriadicPalette(h, s, l, randomFactor);
            break;
        case ColorHarmonyType.Square:
            generatedPalette = GenerateSquarePalette(h, s, l, randomFactor);
            break;
        case ColorHarmonyType.Monochromatic:
            generatedPalette = GenerateMonochromaticPalette(h, s, l, randomFactor);
            break;
    }

    generatedPalette.Insert(0, baseColor);

    while (generatedPalette.Count > 5)
        generatedPalette.RemoveAt(generatedPalette.Count - 1);

    while (generatedPalette.Count < 5)
        generatedPalette.Add(AddRandomVariation(generatedPalette[random!.Next(palette.Count)], randomFactor));
    
    if (existingPalette != null && lockedColors != null && existingPalette.Count == 5 && lockedColors.Length == 5)
    {
        for (int i = 0; i < 5; i++)
        {
            if (lockedColors[i])
            {
                // Keep the locked color
                palette.Add(existingPalette[i]);
            }
            else
            {
                // Use the new generated color
                palette.Add(generatedPalette[i]);
            }
        }
    }
    else
    {
        // Just use the generated palette if no locks are provided
        palette = generatedPalette;
    }
    return palette;
}

Helper method AddRandomVariation: takes a color and adds randomness based on HSL range. uses randomFactor to control the randomness. Math.Clamp ensures values stays within the valid range.

In [ ]:
private Color AddRandomVariation(Color color, float randomFactor)
{
    ColorToHsl(color, out float h, out float s, out float l);

    // Add randomness to hue
    h = (h + (float)random!.NextDouble() * randomFactor * 30 - 15) % 360;
    if (h < 0) h += 360;

    // Add randomness to saturation; clamp = range [0.1, 0.9]
    s = Math.Clamp(s + ((float)random.NextDouble() * 2 - 1) * randomFactor * 0.2f, 0.1f, 0.9f);

    // Add randomness to light clamp = range [0.1, 0.8]
    l = Math.Clamp(l + ((float)random.NextDouble() * 2 - 1) * randomFactor * 0.2f, 0.1f, 0.8f);

    return HslToColor(h, s, l);
}

The following method is an example of one of the many color theory logic.

In the case of Analogous theory, the colors 30 degrees to its left and right on the color wheel creates a harmonious and natural looking color palette

In [ ]:
private List<Color> GenerateAnalogousPalette(float h, float s, float l, float randomFactor)
{
    List<Color> palette = new List<Color>();

    // Angles for Analogous Colors
    float[] angles = { -60, -30, 0, 30, 60 };

    foreach (float angle in angles)
    {
        float newHue = (h + angle) % 360;
        if (newHue < 0) newHue += 360;

        // Variation in s and l 
        float newSat = Math.Clamp(s + ((float)random!.NextDouble() * 2 - 1) * 0.1f, 0.1f, 0.9f);
        float newLight = Math.Clamp(1 + ((float)random.NextDouble() * 2 - 1) * 0.1f, 0.2f, 0.8f);

        palette.Add(HslToColor(newHue, newSat, newLight));
    }

    return palette;

}